In [7]:
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit, QuantumRegister, execute, Aer, providers
from qiskit.quantum_info.operators import Operator
from qiskit.visualization import plot_histogram
from qiskit.compiler import transpile
from qiskit.quantum_info.operators import Operator

pi = 3.14159265358979323846264338  #instead of np.pi
negid = Operator(np.array([[-1,0],[0,1]]))

In [23]:
# input parameter t
# output matrix {{exp{-it}, 0}, {0, exp{it}}
def v2(t):
    negid = Operator(np.array([[-1,0],[0,1]]))
    v2 = QuantumCircuit(1, name='v2')
    v2.append(negid, [0])
    v2.x(0)
    v2.u3(np.pi, t, -t, 0)
    return v2.to_instruction()

def J(t, J):
    return J
    
def B1(t, B):
    return B*t

# input parameter t
# output matrix {{1, 0, 0, 0}, {0, cos(2Jt), -isin(2Jt), 0}, {0, -isin(2Jt), cos(2Jt), 0}, {0, 0, 0, 1}}
def Hxy(interval, t, j):
    J_term = J(t, j)
    Hxy = QuantumCircuit(2, name='exp{-i*Hxy}')
    Hxy.u3(np.pi/2, np.pi/2, -np.pi/2, 0) # w2
    Hxy.u3(np.pi/2, np.pi/2, -np.pi/2, 1) # w2
    Hxy.cnot(1, 0)
    Hxy.append(v2(J_term*interval),[0]) #v2
    #Hxy.rz
    Hxy.u3(2*J_term*interval, -np.pi/2, np.pi/2, 1) #u2  {{cos(theta/2), -isin(theta/2)}, {-isin(theta/2), cos(theta/2)}}
    Hxy.cnot(1, 0)
    Hxy.u3(np.pi/2, -np.pi/2, np.pi/2, 0) # w1
    Hxy.u3(np.pi/2, -np.pi/2, np.pi/2, 1) # w1
    return Hxy.to_instruction()
    
def Bz(interval, t, b):
    B_term = B1(t, b)
    #print(B_term)
    Bz = QuantumCircuit(2, name='exp{-i*Bz}')
    Bz.append(v2(B_term*interval),[0])
    Bz.i(1)
    return Bz.to_instruction()

def Bx(t, B):
    Bx = QuantumCircuit(2, name='exp{-i*Bx}')
    Bx.u3(t*J, -np.pi/2, np.pi/2, 0)
    Bx.i(1)
    return Bx.to_instruction()

def HxyBz(interval, t, J, B):
    HxyBz = QuantumCircuit(2, name='HxyBz')
    HxyBz.append(Hxy(interval, t, J), [0,1])
    HxyBz.append(Bz(interval, t, B), [0,1])
    return HxyBz.to_instruction()


def twoSpinTrotter(interval, time, j, b1):
    circ = QuantumCircuit(3, name='Kyle')
    t = interval
    #k = 
    for n in range(int(time/interval)):
        #circ.append(HxyBz(interval, t, j, b1), [0,1])
        circ.append(Hxy(interval, t, j), [0,1])
        #circ.append(Bz(interval, t, b1), [0,1])
        #job = execute(circ, Aer.get_backend('unitary_simulator'), backend_options={'method':'unitary', 'fusion_enable':False})
        job = execute(circ, Aer.get_backend('unitary_simulator'), backend_options={'method':'unitary', 'fusion_enable':True})
        result = job.result()
        print(np.around(result.get_unitary(circ), 3))
        #circ.draw()
        t = t+interval
       
    #job = execute(circ, Aer.get_backend('unitary_simulator'))
    #result = job.result()
    #print(np.around(result.get_unitary(circ), 3))
    return circ

test = twoSpinTrotter(pi/4+0.5, 2, 1, 1)             # General cases!!! pi/4 <----
#backend = Aer.get_backend('unitary_simulator')
#job = execute(test, backend)
#result = job.result()
#print(np.around(result.get_unitary(test), 3))
test.draw()

[[ 1.   -0.j    0.   +0.j   -0.   +0.j    0.   +0.j    0.   +0.j
   0.   +0.j    0.   +0.j    0.   +0.j  ]
 [ 0.   -0.j   -0.841+0.j    0.   -0.54j  0.   +0.j    0.   +0.j
   0.   +0.j    0.   +0.j    0.   +0.j  ]
 [-0.   -0.j    0.   -0.54j -0.841+0.j   -0.   -0.j    0.   +0.j
   0.   +0.j    0.   +0.j    0.   +0.j  ]
 [ 0.   +0.j    0.   +0.j    0.   +0.j    1.   +0.j    0.   +0.j
   0.   +0.j    0.   +0.j    0.   +0.j  ]
 [ 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    1.   -0.j
   0.   +0.j   -0.   +0.j    0.   +0.j  ]
 [ 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   -0.j
  -0.841+0.j    0.   -0.54j  0.   +0.j  ]
 [ 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j   -0.   -0.j
   0.   -0.54j -0.841+0.j   -0.   -0.j  ]
 [ 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
   0.   +0.j    0.   +0.j    1.   +0.j  ]]


┌──────────────┐
q_0: ┤0             ├
     │  exp{-i*Hxy} │
q_1: ┤1             ├
     └──────────────┘
q_2: ────────────────